## Initialize

In [1]:
from dummy_spark import SparkContext
# from pyspark import SparkContext

In [2]:
from sympy import *
from drudge import *
from agp_fermi import *

In [3]:
ctx = SparkContext()

In [4]:
dr = AGPFermi(ctx)

In [5]:
names = dr.names

In [6]:
p, q, r, s  = names.A_dumms[:4]

## Creation / annihilation operators

In [7]:
c_p = dr.sum(names.c_[p, UP])

In [8]:
cdag_p = dr.sum(names.c_dag[p, DOWN])

In [9]:
(c_p*cdag_p + cdag_p*c_p).display()

<IPython.core.display.Math object>

In [10]:
dr.simplify(c_p*cdag_p + cdag_p*c_p).display()

<IPython.core.display.Math object>

In [11]:
(dr.fermi_dr.dagger(c_p)).display()

<IPython.core.display.Math object>

## Pairing operators

In [12]:
P_p = dr.sum( dr.P[p] )
Pdag_q = dr.sum( dr.Pdag[q] )

In [13]:
Pdag_q.display()

<IPython.core.display.Math object>

In [14]:
P_p.display()

<IPython.core.display.Math object>

In [15]:
expr = dr.simplify(P_p * Pdag_q - Pdag_q * P_p)
expr.display()

<IPython.core.display.Math object>

## Spin flip su(2)

In [16]:
Sp = dr.sum(names.J_p[p])
Sm = dr.sum(names.J_m[q])

In [17]:
Sp.display()

<IPython.core.display.Math object>

In [18]:
expr = dr.simplify( Sp * Sm - Sm * Sp )
expr.display()

<IPython.core.display.Math object>

## Ordering in mixed-algebra expressions

In [19]:
expr2 = Sp * Pdag_q * Sm
expr2.display()

<IPython.core.display.Math object>

In [20]:
expr2.simplify().display()

<IPython.core.display.Math object>

## Unitary group operators

In [21]:
e_ = names.e_
expr = dr.simplify(e_[q,p])
expr.display()

<IPython.core.display.Math object>

## BCS with $c^\dagger$

In [22]:
expr = dr.simplify(cdag_p | Pdag_q)
expr.simplify().display()

<IPython.core.display.Math object>

In [23]:
cdag_p.display()

<IPython.core.display.Math object>

In [24]:
P_q = dr.sum(names.P_[q])
expr = dr.simplify(cdag_p | P_q)
expr.simplify().display()

<IPython.core.display.Math object>

In [25]:
cdag_pup = dr.sum(names.c_dag[p, UP])
expr = dr.simplify(cdag_pup | P_q)
expr.simplify().display()

<IPython.core.display.Math object>

In [26]:
c_pup = c_p
c_pdn = dr.sum(names.c_[p, DOWN])
expr = (c_pup | Pdag_q).simplify()
expr.display()

<IPython.core.display.Math object>

In [27]:
expr = (c_pdn | Pdag_q).simplify()
expr.display()

<IPython.core.display.Math object>

In [28]:
Nq = dr.sum(names.N_[q])
expr = (c_pup | Nq).simplify()
expr.display()

<IPython.core.display.Math object>

## SU2 with $c^\dagger$

In [29]:
Jm_q = dr.sum(names.J_m[q])
Jp_q = dr.sum(names.J_p[q])
Jz_q = dr.sum(names.J_z[q])

In [30]:
(cdag_pup | Jm_q).simplify().display()

<IPython.core.display.Math object>

In [31]:
(cdag_pup | Jz_q).simplify().display()

<IPython.core.display.Math object>

In [32]:
(c_pup | Jz_q).simplify().display()

<IPython.core.display.Math object>

## Utilities for extracting seniority zero components

### Extract the obvious SU2 candidates

#### Number operator example

In [33]:
epp = e_[p,p]

In [34]:
dr.simplify(epp).display()

<IPython.core.display.Math object>

In [37]:
num_op = dr.extract_su2(dr.sum(epp))
num_op.simplify().display()

<IPython.core.display.Math object>

#### Pdag P operator example -- Need for canonicalizing the indices

In [41]:
epq2 = dr.simplify(dr.sum(e_[p,q]*e_[q,p]))
epq2.display()

<IPython.core.display.Math object>

In [42]:
epq2 = dr.canon_indices(epq2)
epq2.display()

<IPython.core.display.Math object>

In [43]:
epq2_bcs = dr.extract_su2(epq2)
epq2_bcs.simplify().display()

<IPython.core.display.Math object>

#### NOTE: Identifying spin-flip operators has been turned off for the time being.

In [ ]:
sp_fermi = dr.simplify(names.c_dag[p, UP] * names.c_[p, DOWN])

In [ ]:
sp_fermi.display()

In [ ]:
dr.extract_su2(sp_fermi).display()

## Nil-potency

In [44]:
expr = cdag_p*cdag_p

In [45]:
expr.display()

<IPython.core.display.Math object>

In [46]:
expr.simplify().display()

<IPython.core.display.Math object>

## *One Function to rule them all*
#### `get_seniority_zero`

In [47]:
c_ = names.c_
c_dag = names.c_dag
expr = dr.sum(c_dag[p, UP] * c_dag[q, DOWN] * c_[r, DOWN] * c_[s, UP])
expr.simplify().display()

<IPython.core.display.Math object>

In [48]:
out_exp = dr.get_seniority_zero(expr)
out_exp.simplify().simplify().merge().display()

<IPython.core.display.Math object>

## AGP-DPQ RPA

#### 1. Pairing Hamiltonian

In [49]:
N = names.N
Pdag = names.P_dag
P = names.P_

In [50]:
eps = IndexedBase('\epsilon')
G = Symbol('G')
H = dr.einst(eps[p]*N[p]) - G*dr.sum((p,names.A), (q, names.A), Pdag[p] * P[q])
Ham_fermi = dr.einst(eps[p]*e_[p,p]) \
    - G*dr.sum((p,names.A), (q, names.A), e_[p,q]*e_[p,q] - KroneckerDelta(p,q)*e_[p,q])/2

In [51]:
H = dr.simplify(H)
H.display()

<IPython.core.display.Math object>

In [52]:
Ham_fermi = dr.simplify(Ham_fermi)
Ham_fermi.display()

<IPython.core.display.Math object>

In [53]:
eta = IndexedBase('\eta')
D_pq = dr.sum(eta[p]*e_[p,q] - eta[q]*e_[q,p])
D_dag_pq = dr.sum(eta[p]*e_[q,p] - eta[q]*e_[p,q])

In [54]:
D_pq.display()

<IPython.core.display.Math object>

In [55]:
D_dag_pq.merge().display()

<IPython.core.display.Math object>

In [56]:
term1 = dr.simplify( Ham_fermi | D_dag_pq )
term1.display()

<IPython.core.display.Math object>

In [57]:
term2 = dr.simplify( D_pq | term1 )
term2.display()

<IPython.core.display.Math object>

### A- Matrix

In [58]:
Amat_pq = dr.simplify( dr.simplify(term2) )

In [59]:
Amat_pq.merge().display()

<IPython.core.display.Math object>

In [60]:
amat_su2 = dr.get_seniority_zero(Amat_pq)
amat_su2 = dr.simplify(amat_su2)
amat_su2.merge().display()

<IPython.core.display.Math object>

### B - Matrix

In [61]:
term1 = dr.simplify(Ham_fermi | D_pq)
term2 = dr.simplify(D_pq | term1)

In [62]:
Bmat_pq = dr.simplify(term2)
Bmat_pq.merge().display()

<IPython.core.display.Math object>

In [63]:
bmat_su2 = dr.get_seniority_zero(-Bmat_pq)
bmat_su2.merge().display()

<IPython.core.display.Math object>

## Expectation value

In [64]:
from conf import *

In [67]:
Amat_expression = dr.eval_agp(amat_su2.merge(), rdm_list)
Amat_expression.simplify().merge().display()

<IPython.core.display.Math object>

In [69]:
Bmat_expression = dr.eval_agp(bmat_su2, rdm_list)
Bmat_expression.simplify().merge().display()

<IPython.core.display.Math object>

-----